<a href="https://colab.research.google.com/github/adolfoguimaraes/inteligenciaartificial/blob/main/code/01_AlgoritmosDeBusca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de Busca 

Nesse notebook vamos trabalhar com a implementação dos três algoritmos de buscas vistos em sala de aula: (1) busca de custo uniforme, (2) busca gulosa e (3) busca A*. Após a implementação do algoritmo vamos aplica-los a dois cenários: o labirinto e o quebra-cabeça 3x3.

In [1]:
# imports necessários

import math 

## A classe `Search`

A classe `Search` implementa os principais componentes dos algoritmos de busca. Vale lembrar que os três algoritmos estudados funcionam da mesma forma, mudando apenas a função que calcula o custo da solução. 

Na **busca de custo uniforme**, o custo $f(x)$ é determinado por $g(x)$ (o custo do caminho). Já na **busca gulosa**, esse custo é determinado por $h(x)$ (o custo da heurística). Por fim, na **busca A***, o custo é determinado por $g(x) + h(x)$. 

O cálculo na classe é feito pela função `cost_function` que recebe como entrada um caminho para uma solução (`path`).

```python
def cost_function(self, path):
        if self.type == 'cost_uniform':
            return path[1]
        elif self.type == 'greedy':
            return self.heuristic(path)
        elif self.type == 'a_star':
            return path[1] + self.heuristic(path)
        else:
            raise BaseException("Algorithm type not allowed. \\
            Choose: (1) cost_uniform, (2) greedy or (3) a_star.")
```

In [2]:
class Search:

    # Inicialização das variáveis
    def __init__(self, type=None):
        self.type = type # Tipo da busca
        self.frontier = None # Fronteira, representada por uma lista de possíveis soluções
        self.best_frontier = None # Melhor solução da fronteira
        self.first_state = None  # Estado inicial da busca
        self.objective_state = None  # Estado objetivo da busca
        self.explored = None # Lista de estados que já foram explorados

    # Função que calcula o custo da solução 
    def cost_function(self, path):
        if self.type == 'cost_uniform':
            return path[1] # No custo uniforme, esse custo é dado pelo custo do caminho
        elif self.type == 'greedy':
            return self.heuristic(path) # No caso da busca gulosa, o custo é dado pelo valor da heurística
        elif self.type == 'a_star':
            return path[1] + self.heuristic(path) # No caso da busca A*, o custo é dado pelo soma do custo do caminho mais a heurística
        else:
            # Erro para quando nenhum dos três tipos de busca é selecionado, 
            raise BaseException("Algorithm type not allowed. Choose: (1) cost_uniform, (2) greedy or (3) a_star.")

    # Função que procura o melhor elemento da fronteira
    def get_best_frontier(self):

        less_value = float('Inf')
        best_frontier_ = []

        for path in self.frontier:
            final_cost = self.cost_function(path)
            if final_cost < less_value:
                less_value = final_cost
                best_frontier_ = path

        
        self.best_frontier = best_frontier_
        
    # Função que imprime o resultado da busca
    def print(self):

        for f in self.frontier:
            if self.best_frontier == f:
                print("* %s : %.1f" % (f[0], self.cost_function(f)))
            else:
                print("%s : %.2f" % (f[0], self.cost_function(f)))

        print("-----------")

    
    # Execução do algoritmo de busca
    def run(self, verbose=False):
        
        self.explored = []
        
        # Cria a fronteira com apenas o estado inicial da busca
        self.frontier = [
            self.first_state
        ]

        # Procura o melhor elemento da fronteira e associa a variável self.best_frontier
        self.get_best_frontier()

        # Atualiza a lista de explorados
        if self.best_frontier[0][-1] not in self.explored:
            self.explored.append(self.best_frontier[0][-1])

        count = 0

        # Se verbose for True imprime os detalhes durante a execução do algoritmo
        if(verbose):
            self.print()

        # Realiza a busca enquanto não atingir o estado objetivo
        while self.best_frontier[0][-1] != self.objective_state:

            # Verifica os possíveis estados a partir do estado atual
            possible_states = self.get_adjacent_not_visited(self.best_frontier[0][-1])
            
            # Remove o melhor elemento da fronteira
            self.frontier.remove(self.best_frontier)

            # Atualiza a fronteira com os novos caminhos
            for state in possible_states:
                new_frontier = ([e for e in self.best_frontier[0]], self.best_frontier[1] + state[1])
                new_frontier[0].append(state[0])
                self.frontier.append(new_frontier)

            # Procura o melhor elemento da fronteira e associa a variável self.best_frontier
            self.get_best_frontier()

            if verbose:
                self.print()

            # Atualiza a lista de explorados
            if self.best_frontier[0][-1] not in self.explored:
                self.explored.append(self.best_frontier[0][-1])

            count += 1


        # Imprime o melhor resultado
        result = [e for e in self.best_frontier[0]]
        count_cost = self.best_frontier[1]

        print("Melhor caminho da fronteira: %s " % result )
        print("Custo do melhor caminho: %s" % count_cost)
        if verbose:
            print("Estados explorados: %s" % self.explored)
        print("Numero de estados explorados: %s" % len(self.explored)) 
        
    
    # A heurística é específica de cada problema. Ela deve ser implementada de acordo com o problema que está sendo resolvido.
    def heuristic(self, path):
        return 0

    # Esse método retorna os estados possíveis a partir do estado atual. Ele também depende do problema e deve ser implementado
    # de acordo com o problema que está sendo resolvido.
    def get_adjacent_not_visited(self, state):
        pass



A classe implementada pode ser usada para resolver diferentes problemas de busca. Para isso é necessário instaciar o problema a partir da classe `Search` e implementar os métodos `heuristic` e `get_adjacent_no_visited`. Aqui será trabalhado dois problemas vistos em sala de aula: o **Problema do Labirinto** e o **Problema do Quebra-Cabeça**. 

## Problema do Labirinto

Esse problema consiste em dado um labirinto, mapeado a partir de um grafo, e um ponto de partida, o algoritmo vai calcular o caminho desse ponto até um ponto pré-definido como _saída do labirinto_. O labirinto mapeado é mostrado na imagem a seguir. 

<img src="../imgs/labirinto1.png" width="50%" />

Esse labirinto é mapeado para o grafo a seguir: 

<img src="../imgs/labirinto2.png" width="50%" />

O labirinto foi mapeado para a estrutura armazena na variável `map_maze`. 

In [3]:
# Classe que implementa o problema do labirinto
class MazeSearch(Search):

    # Inicialização das variáveis
    def __init__(self, type):
        super().__init__(type)

        # Labirinto mapeado
        self.map_maze = {
            'A': {'adjacent': [('B', 5)], 'point': (1, 1)},
            'B': {'adjacent': [('A', 5), ('C', 7), ('F', 2)], 'point': (1, 6)},
            'C': {'adjacent': [('B', 7), ('L', 8)], 'point': (1, 13)},
            'D': {'adjacent': [('E', 3)], 'point': (3, 1)},
            'E': {'adjacent': [('D', 3), ('I', 6)], 'point': (3, 4)},
            'F': {'adjacent': [('B', 2), ('G', 5), ('J', 6)], 'point': (3, 6)},
            'G': {'adjacent': [('F', 5), ('K', 6)], 'point': (3, 11)},
            'H': {'adjacent': [('I', 3)], 'point': (9, 1)},
            'I': {'adjacent': [('E', 6), ('J', 2)], 'point': (9, 4)},
            'J': {'adjacent': [('F', 6), ('I', 2), ('K', 5), ('O', 2)], 'point': (9, 6)},
            'K': {'adjacent': [('G', 6), ('J', 5), ('L', 2), ('T', 9)], 'point': (9, 11)},
            'L': {'adjacent': [('C', 8), ('K', 2), ('U', 9)], 'point': (9, 13)},
            'M': {'adjacent': [('N', 3)], 'point': (11, 1)},
            'N': {'adjacent': [('M', 3), ('O', 2), ('R', 7)], 'point': (11, 4)},
            'O': {'adjacent': [('J', 2), ('N', 2), ('P', 3)], 'point': (11, 6)},
            'P': {'adjacent': [('O', 3), ('S', 7)], 'point': (11, 9)},
            'Q': {'adjacent': [('R', 3)], 'point': (18, 1)},
            'R': {'adjacent': [('N', 7), ('Q', 3), ('S', 5)], 'point': (18, 4)},
            'S': {'adjacent': [('P', 7), ('R', 5), ('T', 2)], 'point': (18, 9)},
            'T': {'adjacent': [('K', 9), ('S', 2), ('U', 2)], 'point': (18, 11)},
            'U': {'adjacent': [('L', 9), ('T', 2)], 'point': (18, 13)}
        }

        # Determina o estado inicial do problema no formato (CAMINHO, CUSTO)
        self.first_state = (['A'], 0)

        # Determina o estado objetivo (saída do labirinto)
        self.objective_state = 'Q'
        

    # Implementação da heurística, que nesse caso é a distância em linha reta de dois pontos.
    def heuristic(self, path ):

        pointA = path[0][-1]
        pointB = self.objective_state 

        pointa_values = self.map_maze[pointA]['point']
        pointb_values = self.map_maze[pointB]['point']

        diff_x = math.pow(pointa_values[0] - pointb_values[0], 2)
        diff_y = math.pow(pointa_values[1] - pointb_values[1], 2)

        distance = math.sqrt(diff_x + diff_y)

        return distance

    # Implementação do método que retorna os vértices adjacentes a partir do grafo do labirinto
    def get_adjacent_not_visited(self, state_):

        states = self.map_maze[state_]['adjacent']
        return_ = []

        for s in states:
            if s[0] not in self.explored:
                return_.append(s)

        return return_



Aplicando o algoritmo utilizando a **busca de custo uniforme**.

In [4]:
search = MazeSearch(type='cost_uniform')
search.run()

Melhor caminho da fronteira: ['A', 'B', 'F', 'J', 'O', 'N', 'R', 'Q'] 
Custo do melhor caminho: 27
Numero de estados explorados: 19


Aplicando o algoritmo utilizando a **busca gulosa**

In [5]:
search = MazeSearch(type='greedy')
search.run()

Melhor caminho da fronteira: ['A', 'B', 'F', 'J', 'O', 'N', 'R', 'Q'] 
Custo do melhor caminho: 27
Numero de estados explorados: 8


Aplicando o algoritmo utilizando a **busca A***

In [8]:
search = MazeSearch(type='a_star')
search.run()

Melhor caminho da fronteira: ['A', 'B', 'F', 'J', 'O', 'N', 'R', 'Q'] 
Custo do melhor caminho: 27
Numero de estados explorados: 10


## Problema do Quebra Cabeça

Esse problema consiste em resolver o quebra-cabeça 3x3 utilizando os algoritmos de busca. A imagem a seguir mostra um exemplo do quebra cabeça embaralhado.

<img src="../imgs/quebracabeca1.png" width="50%" />

O objetivo é aplicar o algoritmo de busca para encontrar a solução: o quebra cabeça montado com as peças nas posições corretas. Como mostrado na imagem a seguir:

<img src="../imgs/quebracabeca2.jpg" width="50%" />

Para resolver o problema, o quebra cabeça foi representado por meio de um array de 9 posições. Nesse array, o 0 indica o espaço vazio e os demais números representam cada peça. 

Por exemplo, a primeira imagem é representada pelo array `[5, 2, 8, 4, 1, 7, 0, 3, 6]`. O objetivo é encontrar o array que represente o quebra-cabeça montado. No caso, seria o array: `[0, 1, 2, 3, 4, 5, 6, 7, 8]`.

In [10]:
# Classe que implementa o problema do Quebra-Cabeça
class PuzzleSearch(Search):

    # Inicialização das variáveis
    def __init__(self, type):
        super().__init__(type)

        # Estado inicial, que representa o quebra cabeça embaralhado, com as peças fora do lugar.
        # O custo dessa solução é 0.
        self.first_state = ([[5, 2, 8, 4, 1, 7, 0, 3, 6]], 0) 

        # Estado final, que representa o quebra cabeça montado, com as peças em seus lugares. 
        # O lugar correto das peças é definido pelo índice no array.
        self.objective_state = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        


    # Implementação da heurística para esse problema.
    # Aquie a heurística corresponde ao total de peças fora do lugar. 
    def heuristic(self, path):

        result = list(i[0] == i[1] for i in zip(self.objective_state, path[0][-1]))
        return 9 - sum(result)

    # Implementação do método que define os próximos estados a partir do estado atual.
    # No quebra-cabeça 3x3 esse estado é definido de acordo com os movimentos possíveis das peças no estado atual.
    def get_adjacent_not_visited(self, state_):

        
        black_index = [index for index, number in enumerate(state_) if number == 0][0]

        index_up = black_index - 3
        index_down = black_index + 3
        index_right = black_index + 1
        index_left = black_index - 1

        possible_states = []

        if index_up >= 0:
            new_path = [e for e in state_]
            temp_value = new_path[index_up]
            new_path[index_up] = 0
            new_path[black_index] = temp_value

            if new_path not in self.explored:
                possible_states.append((new_path, 1))

        if index_down < 9:
            new_path = [e for e in state_]
            temp_value = new_path[index_down]
            new_path[index_down] = 0
            new_path[black_index] = temp_value

            if new_path not in self.explored:
                possible_states.append((new_path, 1))

        if index_right < 9 and index_right % 3 != 0:
            new_path = [e for e in state_]
            temp_value = new_path[index_right]
            new_path[index_right] = 0
            new_path[black_index] = temp_value

            if new_path not in self.explored:
                possible_states.append((new_path, 1))

        if index_left >= 0 and black_index % 3 != 0:
            new_path = [e for e in state_]
            temp_value = new_path[index_left]
            new_path[index_left] = 0
            new_path[black_index] = temp_value

            if new_path not in self.explored:
                possible_states.append((new_path, 1))

        
        return possible_states



Aplicando o algoritmo utilizando a **busca gulosa**.  

In [12]:
search = PuzzleSearch(type='greedy')
search.run()

Melhor caminho da fronteira: [[5, 2, 8, 4, 1, 7, 0, 3, 6], [5, 2, 8, 0, 1, 7, 4, 3, 6], [0, 2, 8, 5, 1, 7, 4, 3, 6], [2, 0, 8, 5, 1, 7, 4, 3, 6], [2, 1, 8, 5, 0, 7, 4, 3, 6], [2, 1, 8, 5, 3, 7, 4, 0, 6], [2, 1, 8, 5, 3, 7, 0, 4, 6], [2, 1, 8, 0, 3, 7, 5, 4, 6], [2, 1, 8, 3, 0, 7, 5, 4, 6], [2, 1, 8, 3, 4, 7, 5, 0, 6], [2, 1, 8, 3, 4, 7, 5, 6, 0], [2, 1, 8, 3, 4, 0, 5, 6, 7], [2, 1, 8, 3, 0, 4, 5, 6, 7], [2, 1, 8, 0, 3, 4, 5, 6, 7], [2, 1, 8, 5, 3, 4, 0, 6, 7], [2, 1, 8, 5, 3, 4, 6, 0, 7], [2, 1, 8, 5, 3, 4, 6, 7, 0], [2, 1, 8, 5, 3, 0, 6, 7, 4], [2, 1, 0, 5, 3, 8, 6, 7, 4], [2, 0, 1, 5, 3, 8, 6, 7, 4], [2, 3, 1, 5, 0, 8, 6, 7, 4], [2, 3, 1, 0, 5, 8, 6, 7, 4], [0, 3, 1, 2, 5, 8, 6, 7, 4], [3, 0, 1, 2, 5, 8, 6, 7, 4], [3, 1, 0, 2, 5, 8, 6, 7, 4], [3, 1, 8, 2, 5, 0, 6, 7, 4], [3, 1, 8, 2, 0, 5, 6, 7, 4], [3, 1, 8, 0, 2, 5, 6, 7, 4], [0, 1, 8, 3, 2, 5, 6, 7, 4], [1, 0, 8, 3, 2, 5, 6, 7, 4], [1, 2, 8, 3, 0, 5, 6, 7, 4], [1, 2, 8, 3, 5, 0, 6, 7, 4], [1, 2, 0, 3, 5, 8, 6, 7, 4], [1, 0, 2, 3, 

Aplicando o algoritmo utilizando a **busca A***.  

In [11]:
search = PuzzleSearch(type='a_star')
search.run()

Melhor caminho da fronteira: [[5, 2, 8, 4, 1, 7, 0, 3, 6], [5, 2, 8, 4, 1, 7, 3, 0, 6], [5, 2, 8, 4, 1, 7, 3, 6, 0], [5, 2, 8, 4, 1, 0, 3, 6, 7], [5, 2, 0, 4, 1, 8, 3, 6, 7], [5, 0, 2, 4, 1, 8, 3, 6, 7], [0, 5, 2, 4, 1, 8, 3, 6, 7], [4, 5, 2, 0, 1, 8, 3, 6, 7], [4, 5, 2, 1, 0, 8, 3, 6, 7], [4, 0, 2, 1, 5, 8, 3, 6, 7], [0, 4, 2, 1, 5, 8, 3, 6, 7], [1, 4, 2, 0, 5, 8, 3, 6, 7], [1, 4, 2, 3, 5, 8, 0, 6, 7], [1, 4, 2, 3, 5, 8, 6, 0, 7], [1, 4, 2, 3, 5, 8, 6, 7, 0], [1, 4, 2, 3, 5, 0, 6, 7, 8], [1, 4, 2, 3, 0, 5, 6, 7, 8], [1, 0, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]] 
Custo do melhor caminho: 18
Numero de estados explorados: 1265


Como esse problema é bem mais custoso que o problema do labirinto. A aplicação da **busca de custo uniforme** fica extremamente custoso.

## É isso 🤖

Estudem as implementações propostas neste notebook para entender como os algoritmos funcionam. Eles podem ser utilizados para resolver outros problemas de busca. 

**Até a próxima aula 🖖**

<img src="https://media.giphy.com/media/3owzVVCtGOpiC6TNdK/giphy.gif" width="50%">

